In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('title_data.tsv', sep='\t', header=0, quoting=3)
print(df.shape)
print(df.head(100))

Cleaning titles

In [ ]:
df['titleType'].unique()

In [ ]:
df=df[~df["titleType"].str.contains('short')]
df=df[~df["titleType"].str.contains('tvEpisode')]
df=df[~df["titleType"].str.contains('tvSpecial')]
df=df[~df["titleType"].str.contains('video')]
df=df[~df["titleType"].str.contains('videoGame')]
df=df[~df["titleType"].str.contains('tvPilot')]

In [ ]:
df['titleType'].unique()

Cleaning ratings

In [ ]:
df_ratings=pd.read_csv('ratings_data.tsv', sep='\t', header=0, quoting=3)
print(df_ratings.shape)
print(df_ratings.head(100))

In [ ]:
df=df.replace('\\N', np.nan)

In [ ]:
df=df.drop(columns=['isAdult','endYear', 'originalTitle'])

In [ ]:
df=pd.merge(df, df_ratings, on='tconst')

In [ ]:
df

In [ ]:
df=df.dropna(subset=['startYear', 'runtimeMinutes', 'genres', 'primaryTitle'])

In [ ]:
#df["genres"].str.find('\\N')
df['genres'].eq('\\N').sum()

In [ ]:
len(df)

In [ ]:
df[df['primaryTitle'].str.contains("Episode")]

In [ ]:
df['genres']=df['genres'].str.split(',').str[0]

In [ ]:
len(df[df['numVotes']<100])

In [ ]:
df = df[df['numVotes'] >= 100]

In [ ]:
df.shape

In [ ]:
df.to_csv('data.tsv', sep="\t", index=False)

In [ ]:
df=pd.read_csv('result.tsv', sep='\t', header=0, quoting=3, lineterminator='\n')
print(df.shape)
print(df.head(100))

In [ ]:
for index, row in df.iterrows():
    if pd.isna(df.iloc[index]['tmdbId']):
        print(index)


In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df[df.isna().any(axis=1)]


In [ ]:
for col in df.columns:
    if col != 'overview':
        print(df[df[col].isna()])


In [ ]:
print(df[df['tmdbId'].isna()])

In [ ]:
df=pd.read_csv('result2.tsv', sep='\t', header=0, quoting=3)
print(df.shape)
print(df.head(100))

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
asd=df[df['tmdbId'].isna()]
asd

In [ ]:
import requests
API_KEY='fa9272e4589b7ec38b742c278e16a2f0'
id='tt9910728'
query = 'https://api.themoviedb.org/3/find/'+id+'?api_key='+API_KEY+'&language=en-US&external_source=imdb_id'
response =  requests.get(query)
movie=response.json()
if len(movie['tv_results']) != 0:
        id=movie['tv_results'][0]['id']
        overview=movie['tv_results'][0]['overview']
        tmdb_vote_avg=movie['tv_results'][0]['vote_average']
        poster=f"https://www.themoviedb.org/t/p/w600_and_h900_bestv2/{movie['tv_results'][0]['poster_path']}"
        
df.loc[df['tconst']=='tt9910728','tmdbId']=id
df.loc[df['tconst']=='tt9910728','overview']=overview
df.loc[df['tconst']=='tt9910728','tmdbVoteAvg']=tmdb_vote_avg
df.loc[df['tconst']=='tt9910728','poster']=poster




In [ ]:
df[df['tmdbId'].isna()]

In [ ]:
df.to_csv('result3.tsv', sep="\t", index=False)

In [ ]:
df=pd.read_csv('./data/result3.tsv', sep='\t', header=0, quoting=3)
print(df.shape)
print(df.head(100))

In [ ]:
df.columns

In [ ]:

#df2 = df[df['tmdbId'] is not np.NaN]
df = df[~df.isnull().any(axis=1)]

In [ ]:
df[df['poster'].isna()]

In [ ]:
df.isnull().any(axis=1)

In [ ]:
old_data=pd.read_csv('./data/Old_800_data.csv', sep=',', header=0, encoding='ISO-8859–1')
old_data = old_data[~old_data.isnull().any(axis=1)]
old_data['title']

In [16]:
import pandas as pd
import numpy as np
df=pd.read_csv('Lili.tsv', sep='\t', header=0, quoting=3)

In [17]:
df.dtypes

tconst             object
titleType          object
primaryTitle       object
startYear         float64
runtimeMinutes    float64
genres             object
averageRating     float64
numVotes          float64
tmdbId            float64
overview           object
tmdbVoteAvg       float64
poster             object
score              object
dtype: object

In [3]:
df = df.replace({'"':''}, regex=True)


In [ ]:
df.to_csv('Lili.tsv', sep="\t", index=False)